In [2]:
# importing libraries
import prices as pr 
import dataio as dtio
from IPython.display import display 
from collections import OrderedDict

%matplotlib inline

## Question 1

### Q1 (i)

Download daily closing price data for S&P 500, FTSE 100 and Gold (SPDR) for the years 2014 to
2017.

In [3]:
# creates directory if it does not exist 
data_path = "data/"

# specify years 2014 to 2017
start_date = "2014-01-01"
end_date = "2018-01-01"

# defining symbols
sp_symbol = "S&P500"
ftse_symbol = "FTSE"
gld_symbol = "GOLD"

# download prices if they are not found in path 
dtio.df2csv(data_path, sp_symbol, pr.download_price, "^GSPC", start_date, end_date)
dtio.df2csv(data_path, ftse_symbol, pr.download_price, "^FTSE?P=FTSE", start_date, end_date) # TODO -> Use online source
dtio.df2csv(data_path, gld_symbol, pr.download_price, "GLD", start_date, end_date) 

# load data from csv files and add to dictionary 
prices_dict = OrderedDict()
prices_dict[sp_symbol] = dtio.csv2df(data_path, sp_symbol)
prices_dict[ftse_symbol] = dtio.csv2df(data_path, ftse_symbol)
prices_dict[gld_symbol] = dtio.csv2df(data_path, gld_symbol)

# print shapes of each symbol and head to make sure data is correct 
for key in prices_dict: 
    print("Symbol: {0} , Shape: {1}".format(key , prices_dict[key].shape))
    display(prices_dict[key].head())

Symbol: S&P500 , Shape: (1008, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-12-31,1842.609985,1849.439941,1842.410034,1848.359985,1848.359985,2312840000
1,2014-01-02,1845.859985,1845.859985,1827.739990,1831.979980,1831.979980,3080600000
2,2014-01-03,1833.209961,1838.239990,1829.130005,1831.369995,1831.369995,2774270000
3,2014-01-06,1832.310059,1837.160034,1823.729980,1826.770020,1826.770020,3294850000
4,2014-01-07,1828.709961,1840.099976,1828.709961,1837.880005,1837.880005,3511750000


Symbol: FTSE , Shape: (300, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-10-24,7020.500000,7067.299805,6983.399902,6986.399902,6986.399902,657275100
1,2016-10-25,6986.399902,7067.200195,6985.899902,7017.600098,7017.600098,836098700
2,2016-10-26,7017.600098,7017.600098,6917.799805,6958.100098,6958.100098,911701300
3,2016-10-27,6958.100098,7003.899902,6924.100098,6986.600098,6986.600098,932928900
4,2016-10-28,6986.600098,7006.399902,6927.200195,6996.299805,6996.299805,927919200


Symbol: GOLD , Shape: (1008, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-12-31,114.610001,117.129997,114.459999,116.120003,116.120003,12835900
1,2014-01-02,117.930000,118.730003,117.750000,118.000000,118.000000,7551000
2,2014-01-03,118.639999,119.620003,118.589996,119.290001,119.290001,5874400
3,2014-01-06,119.760002,120.389999,117.110001,119.500000,119.500000,10106500
4,2014-01-07,118.459999,118.919998,118.129997,118.820000,118.820000,6433700


### Q1 (iii)

Identify the first 4 distribution moments for each index/product mentioned in part (i). For your calculations utilise daily log returns. In your answer describe the calculations/steps performed.

In [3]:
# utilizing the log returns & calculate moment distribtuions
dist_moments = []
for key in prices_dict: 
    # calculating log returns
    prices_dict[key]["log_returns"] = pr.log_dif(prices_dict[key], "Adj Close")
    prices_dict[key] =  prices_dict[key].dropna() 
    
    # calculating moments
    dist_mean, dist_std, dist_skew, dist_kurtosis = pr.dist_moments(prices_dict[key]["log_returns"])
    print("Key: {0}\nMean (1st): {1}\nSTD (2nd): {2}\nSkew (3rd): {3}\nKurtosis (4th): {4} \n"
         .format(key, dist_mean, dist_std, dist_skew, dist_kurtosis))
    
    # append to distribution moments list 
    dist_moments.append((dist_mean, dist_std, dist_skew, dist_kurtosis))    

Key: S&P500
Mean (1st): 0.00036656495314910884
STD (2nd): 0.00762145451287422
Skew (3rd): -0.4096868648378463
Kurtosis (4th): 3.1205740138020275 

Key: FTSE
Mean (1st): 0.00031996402443629115
STD (2nd): 0.005667006414093641
Skew (3rd): -0.21264524446635877
Kurtosis (4th): 1.6712430580998578 

Key: GOLD
Mean (1st): 6.239408654452704e-05
STD (2nd): 0.008785192344062224
Skew (3rd): 0.21891433530118032
Kurtosis (4th): 2.279381345851216 



### Q1 (v)

Annualize daily return (first moment) and volatility (second moment). In your scaling process assume 250 days for the year. In your answer describe the calculations/steps performed.

In [4]:
# annualizing daily returns and volatility 
days_year = 250
dict_keys = list(prices_dict.keys())
for i in range(len(dist_moments)):
    
    # calculating annualized daily return and volatility
    annualized_return = dist_moments[i][0] * days_year 
    annualized_volatility = dist_moments[i][1] * days_year 
    
    # display values
    print("Key: {0} \nAnnualized Returns: {1}\nAnnualized Volatility: {2}\n"
          .format(dict_keys[i], round(annualized_return * 100, 4), round(annualized_volatility * 100, 4)))

Key: S&P500 
Annualized Returns: 9.1641
Annualized Volatility: 190.5364

Key: FTSE 
Annualized Returns: 7.9991
Annualized Volatility: 141.6752

Key: GOLD 
Annualized Returns: 1.5599
Annualized Volatility: 219.6298



### Q1 (vi)

By considering the last closing price at the end of 2017, and the annualized volatility from question (v), what would be the price level of S&P 500 after 1 month, that according to normal probability, there is a 32% chance that the actual price will be above it. Show your workings.

In [1]:
# getting the last close price for 2017 S&P500
days_month = 20
sp_last_close = prices_dict["S&P500"].tail(1)["Adj Close"].values[0]
sp_deviation = sp_last_close * (dist_moments[0][1] * 100) * ((days_month / days_year) ** 0.5)

print("Close: ${0}\nDeviation: ${1}"
      .format(round(sp_last_close, 2), round(sp_deviation,2)))

# https://www.optionsanimal.com/using-implied-volatility-determine-expected-range-stock/
    
# SHOW DISTRIBUTION AND DEVIATIONS TO SHOW THERE IS A 32% CHANCE
print("{0}% chance that S&P500 will be between ${1} and ${2} after one month (January)"
      .format(32, round((sp_last_close - sp_deviation),2), round((sp_last_close + sp_deviation),2)))

# TODO -> Show the 32% thingy 

NameError: name 'prices_dict' is not defined

### Q1 (vii)

Download the Google and Amazon daily prices for the last 5 years (till 31/12/2017). By utilizing a regression model, perform the Beta-test against the S&P 500 index. Comment on your findings.

In [6]:
# specify years 2012 to 2017 (5 years)
start_date = "2012-01-01"
end_date = "2017-12-31"

# defining symbols
google_symbol = "GOOGL"
amazon_symbol = "AMZN"

# download prices if they are not found in path 
dtio.df2csv(data_path, google_symbol, pr.download_price, google_symbol, start_date, end_date)
dtio.df2csv(data_path, amazon_symbol, pr.download_price, amazon_symbol, start_date, end_date)

# load data from csv files and add to dictionary 
prices_dict[google_symbol] = dtio.csv2df(data_path, google_symbol)
prices_dict[google_symbol] = dtio.csv2df(data_path, amazon_symbol)